[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/floleuerer/ml-tutorials/blob/main/04_fastai_text.ipynb)

# Vorbereitung

Laufzeit-Typ ändern. Python-Pakete von fast.ai installieren und importieren.

In [14]:
!pip install -Uq fastai

OSError: [Errno 12] Cannot allocate memory

In [2]:
from fastai.text.all import *

Wichtig! Wenn die folgende Zelle "False" ausgibt, ist keine GPU vorhanden -> Bitte prüfe die Vorbereitungsschritte in Notebook 01 noch einmal und starte ggf. das Notebook neu. Bei "True" hast du eine GPU-Instanz und kannst weiter machen.

In [3]:
torch.cuda.is_available()

True

# Text Klassifizierung

Deep Learning hat sich nicht nur im Bereich Computer Vision, sondern auch im Bereich des Natural Language Processing durchgesetzt. Ein Beispiel dafür ist die Text Klassifizierung und ein beliebter Anwendungsfall z. B. die Sentiment Analyse. Diese Analysiert die Stimmung (positiv / negativ) in Texten.

Im Marketing gehört die Sentiment-Analyse vor allem in den Bereich des Social-Media-Monitorings. Der Begriff Social Media bezieht sich hier nicht nur auf Facebook, Twitter und Co., sondern auch auf YouTube, Produkt- und Dienstleistungsbewertungen in Shops und Portalen sowie Foreneinträge. Über diese Kanäle werden heute nämlich sehr schnell Meinungen gebildet und verbreitet. Wer also weiß, wie in den Kommentaren und Posts über das eigene Produkt geredet wird, der kann entsprechend reagieren.

# Daten laden

Für die Textklassifizierung nutzen wir den IMDb Datensatz aus dem Paper *Learning Word Vectors for Sentiment Analysis*, der ebenfalls in den Standard-Datensets von Fast.ai enthalten ist. Der Datensatz umfasst mehrere Tausend englischsprachige Filmrezensionen.

In [4]:
path = untar_data(URLs.IMDB)

Der Ordner mit den Trainingsdaten enthält zwei weitere Unterordner *pos* und *neg* für positive und negative Rezensionen.

In [5]:
(path/'train').ls()

(#4) [Path('/home/jupyter/.fastai/data/imdb/train/labeledBow.feat'),Path('/home/jupyter/.fastai/data/imdb/train/neg'),Path('/home/jupyter/.fastai/data/imdb/train/pos'),Path('/home/jupyter/.fastai/data/imdb/train/unsupBow.feat')]

`valid_name` gibt den Namen des Ordners an, in dem die Daten zur Validierung liegen

In [6]:
imdb = DataBlock(blocks=(TextBlock.from_folder(path), CategoryBlock),
                 get_items=get_text_files,
                 get_y=parent_label,
                 splitter=GrandparentSplitter(valid_name='test'))

In [7]:
dls = imdb.dataloaders(path)

In [8]:
dls.show_batch()

,text,category
0,"xxbos xxmaj match 1 : xxmaj tag xxmaj team xxmaj table xxmaj match xxmaj bubba xxmaj ray and xxmaj spike xxmaj dudley vs xxmaj eddie xxmaj guerrero and xxmaj chris xxmaj benoit xxmaj bubba xxmaj ray and xxmaj spike xxmaj dudley started things off with a xxmaj tag xxmaj team xxmaj table xxmaj match against xxmaj eddie xxmaj guerrero and xxmaj chris xxmaj benoit . xxmaj according to the rules of the match , both opponents have to go through tables in order to get the win . xxmaj benoit and xxmaj guerrero heated up early on by taking turns hammering first xxmaj spike and then xxmaj bubba xxmaj ray . a xxmaj german xxunk by xxmaj benoit to xxmaj bubba took the wind out of the xxmaj dudley brother . xxmaj spike tried to help his brother , but the referee restrained him while xxmaj benoit and xxmaj guerrero",pos
1,"xxbos xxmaj some have praised _ xxunk _ as a xxmaj disney adventure for adults . i do n't think so -- at least not for thinking adults . \n\n xxmaj this script suggests a beginning as a live - action movie , that struck someone as the type of crap you can not sell to adults anymore . xxmaj the "" crack staff "" of many older adventure movies has been done well before , ( think _ the xxmaj dirty xxmaj dozen _ ) but _ atlantis _ represents one of the worse films in that motif . xxmaj the characters are weak . xxmaj even the background that each member trots out seems stock and awkward at best . xxmaj an xxup md / xxmaj medicine xxmaj man , a tomboy mechanic whose father always wanted sons , if we have not at least seen these before",neg
2,"xxbos xxmaj raising xxmaj victor xxmaj vargas : a xxmaj review \n\n xxmaj you know , xxmaj raising xxmaj victor xxmaj vargas is like sticking your hands into a big , steaming bowl of oatmeal . xxmaj it 's warm and gooey , but you 're not sure if it feels right . xxmaj try as i might , no matter how warm and gooey xxmaj raising xxmaj victor xxmaj vargas became i was always aware that something did n't quite feel right . xxmaj victor xxmaj vargas suffers from a certain overconfidence on the director 's part . xxmaj apparently , the director thought that the ethnic backdrop of a xxmaj latino family on the lower east side , and an idyllic storyline would make the film critic proof . xxmaj he was right , but it did n't fool me . xxmaj raising xxmaj victor xxmaj vargas is",neg
3,"xxbos i thought that xxup rotj was clearly the best out of the three xxmaj star xxmaj wars movies . i find it surprising that xxup rotj is considered the weakest installment in the xxmaj trilogy by many who have voted . xxmaj to me it seemed like xxup rotj was the best because it had the most profound plot , the most suspense , surprises , most xxunk the ending ) and definitely the most episodic movie . i personally like the xxmaj empire xxmaj strikes xxmaj back a lot also but i think it is slightly less good than than xxup rotj since it was slower - moving , was not as episodic , and i just did not feel as much suspense or emotion as i did with the third movie . \n\n xxmaj it also seems like to me that after reading these surprising reviews that",pos
4,"xxbos xxmaj the premise of this movie has been tickling my imagination for quite some time now . xxmaj we 've all heard or read about it in some kind of con - text . xxmaj what would you do if you were all alone in the world ? xxmaj what would you do if the entire world suddenly disappeared in front of your eyes ? xxmaj in fact , the last part is actually what happens to xxmaj dave and xxmaj andrew , two room - mates living in a run - down house in the middle of a freeway system . xxmaj andrew is a nervous wreck to say the least and xxmaj dave is considered being one of the biggest losers of society . xxmaj that alone is the main reason to why these two guys get so well along , because they simply only have each",pos
5,"xxbos xxmaj my xxmaj comments for xxup vivah : - xxmaj its a charming , idealistic love story starring xxmaj shahid xxmaj kapoor and 

Die Texte wurden automatisch von Fast.ai in Tokens aufgeteilt und um einige spezielle Tokens ergänzt. 

`xxbos ` zeigt den Anfang eines Textes

`xxmaj` signalisiert, dass das nächste Wort mit einem Großbuchstaben anfängt.

# Model trainieren

In [9]:
learn = text_classifier_learner(dls, AWD_LSTM, drop_mult=0.5, metrics=accuracy)

In [13]:
learn.fine_tune(4, 1e-2)

epoch,train_loss,valid_loss,accuracy,time


OSError: [Errno 12] Cannot allocate memory

In [ ]:
learn.show_results()

# "predicten"

Probiere hier gerne auch andere Rezensionen aus.

In [ ]:
learn.predict("The movie is really great")

Das Modell sagt über die Rezension, dass sie positiv ist. `tensor(1)` gibt den Index von *pos* im Vokabular an und der letzte Teil die Wahrscheinlichkeiten zu jeder Klasse. (99% *pos* und 0.9% *neg*)